# ECS7024: Introduction to Bayesian Statistics

*Version 2.0*

This notebook accompanies the introduction to Bayesian Statistics (topic 18). The aim of this topic is to gain awareness of Bayesian statistics. It is not expected that students are able to run these analyses; you will not be asked to do this for a coursework. Except for Section 3, the notebook will not run on jhub.

If you do wish to run this notebook you need to install the following libraries:

  * [Pymc3: Probabilistic Programming in Python](https://docs.pymc.io/)
  * [ArviZ: Exploratory analysis of Bayesian models](https://arviz-devs.github.io/arviz/)


**Contents**
  1. Section 1: Binomial Distribution
     1.  Section 1.1: Likelihood
     2.  Section 1.2: Prior
     3.  Section 1.3: Calculating the Posterior
  2. Section 2: Regression Model
     1.  Section 2.1: Simulating Data
     2.  Section 2.2: Specifying the Model
     3.  Section 2.3: Displaying the Posteriors
  3. Section 3: Binomial Again: Direct Implementation
     1.  Section 3.1: The Algorithm
     2.  Section 3.2: Examples: Varying the Prior
     3.  Section 3.2: Examples: Zero Successes

In [1]:
# !pip install pymc3
# !pip install arviz

In [2]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import pandas as pd
import scipy.stats as stats

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
The installed Theano(-PyMC) version (1.0.4) does not match the PyMC3 requirements.
It was imported from ['C:\\Users\\ellio\\anaconda3\\lib\\site-packages\\theano']
For PyMC3 to work, a compatible Theano-PyMC backend version must be installed.
See https://github.com/pymc-devs/pymc3/wiki for installation instructions.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


AttributeError: 'TheanoConfigParser' object has no attribute 'gcc__cxxflags'

In [ ]:
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)

## Section 1: Binomial Distribution 
In this section we look at a binomial distribution. Recall that the parameters of a binomial distribution are:
  1. The number of trials (n)
  2. The probability of success (p)

and that the distribution gives the probability of the number of successes, in the range 0 to n. Imagine that we have gathered data and know both n (the number of trials) and the number of successes. What is p (the probability of success)?



### Section 1.1 Likelihood

In this simple case we can evaluate the probability of the evidence directly. Suppose that the number of trails (n) is 100 and the number of successes (x) is 25. So the usual estimate is 25/100.

We see that the probabilities are quite small. Seeing numbers of successes that are close (such as 23, 24, 2,6, 27) is just as likely, even if the true probability of success is 25%. 

In [ ]:
x = 25 # successes
n = 100 # trials
evidence_prob = pd.DataFrame(data = {'Param': np.arange(0.1,0.5,0.01)})
evidence_prob = evidence_prob.assign(Prob = evidence_prob.apply(lambda row: stats.binom.pmf(x, n, row.Param), axis=1))
evidence_prob = evidence_prob.assign(Log = evidence_prob.apply(lambda row: np.log(row.Prob), axis=1))
evidence_prob[10:20]

**Likelihood Function** This shows the probability of getting the data that we observed varies with the *success probabiliy* which is the parameter of the binomial distribution. We plot this with and without logs.

Note that **neither** plot is a probability distribution. In $p(\mbox{data} ~|~ \theta)$ we have a probability distribution over the different possible data values for a given $\theta$. However, in these plots, it is $\theta$ that varies which the data observed stays the same.

In [ ]:
fig, (a1, a2) = plt.subplots(1,2,figsize=(12, 6))
evidence_prob.plot(kind = "line", y="Prob", x = "Param", ax=a1)
evidence_prob.plot(kind = "line", y="Log", x = "Param", ax=a2)
a1.set_xlabel("Probability of Success")
a2.set_xlabel("Probability of Success")
a1.set_ylabel("Probability of observing 25 successes in 100 trials")
_ = a2.set_ylabel("Log of Prob of observing 25 successes in 100 trials")

### Section 1.2: Prior Probability

We need to specify a prior distribution for the unknown parameter, the probability of success. The beta distribution is often used for this, because the Bayesian formula has an analytic solution in this case. Although this is not relevant to use, we can still use the beta distribution: as it ranges the interval 0 to 1, it is a suitable way to specify a prior for a probability.

The following disagram shows some beta distributions for different parameters. We see that Beta(1, 1) is a uniform distribution on the interval 0 to 1. 

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

def meanBeta(a, b): return a / (a + b)
def modeBeta(a, b): return (a - 1) / (a + b - 2)

x = np.linspace(stats.beta.ppf(0.01, 1, 1), stats.beta.ppf(0.99, 1, 1), 100)
ax.plot(x, stats.beta.pdf(x, 1, 1), 'r-', label=('beta(1, 1), mean = %3.2f' % meanBeta(1, 1)))
ax.plot(x, stats.beta.pdf(x, 1.5, 1.5), 'k-', label=('beta(1.5, 1.5), mean = %3.2f, mode = %3.2f' % (meanBeta(1.5, 1.5), modeBeta(1.5, 1.5))))
ax.plot(x, stats.beta.pdf(x, 3, 7), 'b-', label=('beta(3, 7), mean = %3.2f, mode = %3.2f' % (meanBeta(3, 7), modeBeta(3, 7))))
ax.legend()

### Section 1.3: Calculating the Posterior
Recall the Bayesian formula:
$$
p(\theta \mid \mbox{data}) = \frac{p(\mbox{data} \mid \theta) . p(\theta)}{p(\mbox{data})}
$$
where $\theta$ stands for all the uncertian parameters of the model. In this example, there is only one such parameter but often (as in the second example below) there are many parameters. So we are, in general, seeking to represent a multivariate probability distribution.

The representation that is used is a list of samples. Random (so called 'Monte Carlo') methods are used to create the samples. 


In [ ]:
print(f"Running on PyMC3 v{pm.__version__}")

**Model Specification in PyMC3**

The first step is to specify the model. First the model is introduced and then the variables are added. We have added two variables using different priors. It takes several seconds to generate the samples.

In [ ]:
binom_model = pm.Model()

with binom_model:

    # Priors for unknown model parameters
    p1 = pm.Beta("p1", alpha=1.0, beta=1.0)
    p2 = pm.Beta("p2", alpha=3.0, beta=7.0)
    
    # Likelihood (sampling distribution) of observations
    NSucc1 = pm.Binomial("NSucc1", p=p1, n=100, observed=25)
    NSucc2 = pm.Binomial("NSucc2", p=p2, n=100, observed=25)
    
    # draw posterior samples
    trace = pm.sample(2000, return_inferencedata=False)

The distribution can be plotted.

In [ ]:
with binom_model:
    az.plot_dist(trace["p1"], color="r", label="Uniform prior")
    ax = az.plot_dist(trace["p2"], color="b", label="Prior beta(3, 7)")
    ax.set_xlabel("Success Probability", fontsize=14)
    ax.set_ylabel("Density", fontsize=14)
    

Another form of plot shows the range.

In [ ]:
with binom_model:
    az.plot_forest(trace, var_names=['p1', 'p2'])

## Section 2: Linear Regression

This section demonstrates Bayesian inference for a regression model. 

### Section 2.1 Simulating Data
Data for the regression is simulated by setting some regression coefficient and an intercept and then simulating data from the regression equation plus a normally-distributed random offset.

In [ ]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma

**Residuals**
We can plot the residuals in the simulated data.

In [ ]:
Y_hat = alpha + beta[0]*X1 + beta[1]*X2

fig, ax = plt.subplots()
ax.plot(Y, Y_hat, lw=0.0, ms=2.0, marker='x')
ax.set_xlabel("True Value")
_ = ax.set_ylabel("Predicted Value")

ys = np.arange(-2.0, 5.0, 0.1)
ax.plot(ys, ys)

### Section 2.2 Specifying the Model

The model is specified using 4 variables.

In [ ]:
regression = pm.Model()

with regression:

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10, shape=2)
    sigma = pm.HalfNormal('sigma', sd=1)

    # Expected value of outcome
    mu = alpha + beta[0]*X1 + beta[1]*X2

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=Y)
    
    trace = pm.sample(2000, return_inferencedata=False)

### Section 2.3 Displaying the Posterior

The following plot shows the full trace, with the density plots of the intercept (alpha), the two coeffecients (beta) and the variance of the error (sigma). This is followed by a forest plot. 

All the values are close to the true parameter values used in the model specification.

In [ ]:
with regression:
    az.plot_trace(trace, figsize=(12,12))

In [ ]:
with regression:
    az.plot_forest(trace, var_names=['alpha', 'beta'])

## Section 3: Binomial Again: Direct Implementation
In this section we look again at a binomial distribution, but now do the calculations directly, without using pyMC. 

We want to estimate the probability $p$ of a binomial after a given number of trials $n$ with $x$ successes. In the following equation:
$
   p(\theta \mid D) \propto  p(D \mid \theta) . p(\theta)
$
 *  $\theta$ is the unknown parameter of a probability distribution, to be estimated from data
 *  $D$ is the data, i.e. the results of observation or experiment


The steps to sample from the posterior distribution are:
 1. Sample $\theta$ from prior distribution (e.g uniform)
 2. Calculate the likelihood of the data using the sampled $\theta$
 3. Normalize the weights
 4. Plot a histogram of weighted samples

### Section 3.1: The Algorithm

We create a data frame with the following variables:
  1. theta: the unknown parameter(s) to be inferred
  2. weight: the likelihood weightings

The theta values are sampled from the prior distribution. The likelihood is calculated using the PMF of the binomial.

In [ ]:
# binomial proportion posterior distribution
def binPropDist(trials, success, prior, samples=50000):
    df = pd.DataFrame()
    df = df.assign(theta = prior.rvs(samples))
    df = df.assign(weight = 
                   df.apply(lambda r: stats.binom.pmf(success, trials, r.theta), 
                            axis=1))
    df.weight = df.weight / np.sum(df.weight)
    return df

def plotBinPropDist(df, text, lower=0, upper=1):
    fg, a1 = plt.subplots(1, 1, figsize=(10, 6))
    df1 = df.loc[(df.theta >= lower) & (df.theta <= upper)]
    bins=int(np.ceil(2*np.power(len(df1), 1/3)))
    df1.hist(column='theta', weights=df1.weight, bins=bins, density=True, ax=a1)
    a1.set_title('Posterior Distribution of p, for %s' % text)
    a1.set_xlabel('Unknown Parameter Binomial Proportion p')
    a1.set_ylabel('Density')
    return a1


### Section 3.2: Examples: Varying the Prior
These examples use $n=10$ and $x=2$ with different prior distributions:

  * Uniform
  * Beta

In [ ]:
df = binPropDist(10, 2, stats.uniform())

In [ ]:
ax = plotBinPropDist(df, "2 Successes in 10 Trials, Uniform Prior")
x = np.linspace(0, 1, 100)
ax.plot(x, stats.uniform.pdf(x), 'r-', lw=3, alpha=0.6, label='beta pdf')

In [ ]:
df_beta = binPropDist(10, 2, stats.beta(2, 5))

In [ ]:
a1 = plotBinPropDist(df_beta, "2 Successes in 10 Trial, prior Beta(2,5)")
x = np.linspace(0, 1, 100)
a1.plot(x, stats.beta.pdf(x, 2, 5), 'r-', lw=3, alpha=0.6, label='beta pdf')

In [ ]:
df_beta2 = binPropDist(10, 2, stats.beta(2, 2))
a1 = plotBinPropDist(df_beta2, "2 Successes in 10 Trial, prior Beta(2,2)")
x = np.linspace(0, 1, 100)
a1.plot(x, stats.beta.pdf(x, 2, 2), 'r-', lw=3, alpha=0.6, label='beta pdf')

### Section 3.3: Examples: Zero Successes
We explore this for different numbers of trials:

 1. 10 trials
 2. 100 trials
 3. 500 trials


All cases use a uniform prior

In [ ]:
df_uni1 = binPropDist(10, 0, stats.uniform())
_ = plotBinPropDist(df_uni1, "0 Success in 10 Trials, Uniform Prior", 
                    lower=0, upper=0.5)

In [ ]:
df_uni2 = binPropDist(100, 0, stats.uniform())
_ = plotBinPropDist(df_uni2, "0 Success in 100 Trials, Uniform Prior", 
                    lower=0, upper=0.1)

In [ ]:
df_uni2 = binPropDist(500, 0, stats.uniform(), samples=100000)
_ = plotBinPropDist(df_uni2, "0 Success in 500 Trials, Uniform Prior", 
                    lower=0, upper=0.02)